In [1]:
from sqlalchemy.ext.asyncio import async_sessionmaker, create_async_engine
import sqlalchemy as sqla
from config import env


engine = create_async_engine(
    sqla.URL.create(
        host=env.DB_URL,
        drivername=env.DB_DRIVER,
        username="postgres",
        password="postgres",
        # port=env.DB_PORT,
        port=5432,
        database=env.DB_NAME,
    ),
    pool_pre_ping=True,
)

ss = async_sessionmaker(
    engine,
    expire_on_commit=False,
)()


In [3]:
from db.postgresql import init_db

await init_db()

In [2]:
from db.postgresql.models.product import (
    Product,
    ProductPriceHistory,
    ProductStockHistory,
)
from tqdm.autonotebook import tqdm
from datetime import datetime
import datetime as dt

/tmp/ipykernel_11169/2328573263.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
MAX_PRICE_CHANGE_AMOUNT = 30
MIN_PRICE_CHANGE_AMOUNT = 5

MAX_STOCK_CHANGE_AMOUNT = 30
MIN_STOCK_CHANGE_AMOUNT = 5

MAX_ITEM_PER_ORDER = 5
MIN_ITEM_PER_ORDER = 3

MAX_AMOUNT_OF_ITEM_IN_ORDER = 5
MIN_AMOUNT_OF_ITEM_IN_ORDER = 3

MAX_PRICE = 40
MIN_PRICE = 5

MAX_SALE = 30
MIN_SALE = 0

MAX_PRICE_CHANGE_INTERVAL_DAY = 70
MIN_PRICE_CHANGE_INTERVAL_DAY = 20


MAX_RESTOCK_INTERVAL = 70
MIN_RESTOCK_INTERVAL = 20

In [3]:
USER_AMOUNT = 100

STAFF_AMOUNT = 10
SHIPPER_AMOUNT = 10

MAX_ORDER_PER_USER = 10
MIN_ORDER_PER_USER = 2

MAX_ORDER_INTERVAL_DAY = 14
MIN_ORDER_INTERVAL_DAY = 2

In [5]:
import random
from passlib.context import CryptContext
import faker

from db.postgresql.models.user_account import UserAccount, UserAccountStatus

fake = faker.Faker()

In [6]:
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")


def generate_phone() -> str:
    num_arr = ["0"]

    for _ in range(9):
        num_arr.append(str(random.randint(0, 9)))

    return "".join(num_arr)


user_password = pwd_context.hash("password")

(trapped) error reading bcrypt version
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/passlib/handlers/bcrypt.py", line 620, in _load_backend_mixin
    version = _bcrypt.__about__.__version__
AttributeError: module 'bcrypt' has no attribute '__about__'


In [ ]:
import uuid

pbar = tqdm(desc="Adding customer account", total=USER_AMOUNT)

async with ss.begin():
    for _ in range(USER_AMOUNT):
        username = fake.user_name()
        id = str(uuid.uuid4())

        staff = UserAccount(
            id=id,
            email=fake.email(),
            username=username,
            profile_name=username,
            password=user_password,
            token=username,
            status=random.choices(
                [UserAccountStatus.NORMAL, UserAccountStatus.BANNED],
                [0.9, 0.1],
            )[0],
            address=fake.address(),
            profile_pic_uri="defaultProfile",
            profile_description=" ",
            phone=generate_phone(),
            bookmarked_posts=[""],
        )

        pbar.update()

        ss.add(staff)

    await ss.flush()


Adding customer account:   0%|          | 0/100 [00:00<?, ?it/s]

In [12]:
import uuid

from db.postgresql.models.staff_account import (
    AccountStatus,
    AccountType,
    EmployeeInfo,
    StaffAccount,
)


async def generate_staff(type: AccountType, amount: int):
    pbar = tqdm(desc="Adding customer account", total=amount - 1)

    async with ss.begin():
        for _ in range(amount):
            username = fake.user_name()
            id = str(uuid.uuid4())

            staff = StaffAccount(
                id=id,
                username=username,
                password=user_password,
                type=type,
                token=username,
                status=random.choices(
                    [
                        AccountStatus.ACTIVE,
                        AccountStatus.DISABLE,
                    ],
                    [0.9, 0.1],
                )[0],
            )

            info = EmployeeInfo(
                account_id=id,
                ssn=generate_phone(),
                phonenumber=generate_phone(),
                realname=fake.name(),
                email=fake.email(),
                dob=fake.date_of_birth(),
            )

            pbar.update()

            ss.add(staff)
            ss.add(info)

        await ss.flush()

In [ ]:
await generate_staff(AccountType.STAFF, STAFF_AMOUNT)


Adding customer account:   0%|          | 0/9 [00:00<?, ?it/s]

In [8]:
async with ss.begin():
    product_amount = await ss.scalar(sqla.select(sqla.func.count(Product.id))) or 0
    product_ids = (await ss.scalars(sqla.select(Product.id))).all()
    account_ids = (await ss.scalars(sqla.select(UserAccount.id))).all()
    account_amount = (
        await ss.scalar(
            sqla.select(
                sqla.func.count(
                    UserAccount.id,
                )
            )
        )
        or 0
    )

In [15]:
async with ss.begin():
    pbar_prod = tqdm(total=product_amount - 1)
    pbar_sub = tqdm()

    current_day = datetime.now()

    for prod in product_ids:
        pbar_prod.set_description(f"Product : {prod}")

        price_amount = random.randint(
            MIN_PRICE_CHANGE_AMOUNT,
            MAX_PRICE_CHANGE_AMOUNT,
        )
        stock_amount = random.randint(
            MIN_STOCK_CHANGE_AMOUNT,
            MAX_STOCK_CHANGE_AMOUNT,
        )

        pbar_sub.reset(price_amount)
        pbar_sub.set_description("Generate price")

        prev_date = current_day
        for _ in range(price_amount):
            price = round(random.uniform(MIN_PRICE, MAX_PRICE), 2)
            sale_percent = round(random.uniform(MIN_SALE, MAX_SALE), 2)

            day_offset = dt.timedelta(
                days=random.randint(
                    MIN_PRICE_CHANGE_INTERVAL_DAY, MAX_PRICE_CHANGE_INTERVAL_DAY
                )
            )

            prev_date -= day_offset

            price_history = ProductPriceHistory(
                product_id=prod,
                date=prev_date,
                price=price,
                sale_percent=sale_percent,
            )

            ss.add(price_history)

            pbar_sub.update()

        pbar_sub.reset(stock_amount)
        pbar_sub.set_description("Generate stock")
        prev_date = current_day
        for _ in range(stock_amount):
            price = round(random.uniform(MIN_PRICE, MAX_PRICE) * 0.7, 2)
            stock = random.randint(
                MIN_STOCK_CHANGE_AMOUNT,
                MAX_STOCK_CHANGE_AMOUNT,
            )
            day_offset = dt.timedelta(
                days=random.randint(MIN_RESTOCK_INTERVAL, MAX_RESTOCK_INTERVAL)
            )

            prev_date -= day_offset
            stock = ProductStockHistory(
                product_id=prod,
                in_stock=stock,
                in_price=price,
                date=prev_date,
            )

            ss.add(stock)

            pbar_sub.update()

        pbar_prod.update()

    await ss.flush()

  0%|          | 0/22 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [9]:
import uuid
from db.postgresql.models.order_history import (
    OrderHistory,
    OrderHistoryItems,
    OrderStatus,
    PaymentStatus,
)
from db.postgresql.models.transaction import PaymentTransaction

pbar = tqdm(total=account_amount - 1, desc="Generate per user")
pbar_sub = tqdm(desc="Order:")
async with ss.begin():
    for i in account_ids:
        order_amount = random.randint(MIN_ORDER_PER_USER, MAX_ORDER_PER_USER)

        current_day = datetime.now()

        prev_date = current_day

        pbar_sub.reset(order_amount - 1)
        for _ in range(order_amount):
            item_for_order = random.randint(
                MIN_ITEM_PER_ORDER,
                MAX_ITEM_PER_ORDER,
            )

            day_offset = dt.timedelta(
                days=random.randint(
                    MIN_ORDER_INTERVAL_DAY,
                    MAX_ORDER_INTERVAL_DAY,
                )
            )

            prev_date -= day_offset

            prods = list(
                (
                    await ss.execute(
                        sqla.select(ProductPriceHistory.product_id)
                        .filter(ProductPriceHistory.date < prev_date)
                        .distinct(ProductPriceHistory.product_id)
                    )
                ).scalars()
            )

            if len(prods) < item_for_order:
                break

            order_items = random.sample(prods, item_for_order)

            total_price = 0

            order_id = uuid.uuid4()

            order_items_db = []
            for item in order_items:
                quantity = random.randint(1, 5)
                prod_price = await ss.scalar(
                    sqla.select(ProductPriceHistory)
                    .filter(
                        ProductPriceHistory.product_id == item,
                        ProductPriceHistory.date < prev_date,
                    )
                    .order_by(ProductPriceHistory.date.desc())
                    .limit(1)
                )

                if not prod_price:
                    raise Exception()

                total_price += (prod_price.price * prod_price.sale_percent) * quantity

                order_items_db.append(
                    OrderHistoryItems(
                        order_history_id=order_id,
                        product_id=prod_price.product_id,
                        date=prod_price.date,
                        quantity=quantity,
                    )
                )

            total_price = round(total_price, 2)
            order = OrderHistory(
                id=order_id,
                user_id=i,
                order_date=prev_date,
                delivery_address=fake.address(),
                note=fake.sentence(5),
                total_price=total_price,
                receiver=fake.name(),
                phonenumber=fake.phone_number(),
                coupon=None,
                payment_method="COD",
                payment_status=PaymentStatus.PENDING,
                order_status=random.choice([
                    OrderStatus.CANCELLED,
                    OrderStatus.ON_CONFIRM,
                    OrderStatus.ON_PROCESSING,
                    OrderStatus.ON_SHIPPING,
                    OrderStatus.SHIPPED,
                ]),
            )

            payment = PaymentTransaction(
                order_id=order.id,
                create_time=order.order_date,
                amount=order.total_price,
                status=PaymentStatus.RECEIVED,
            )

            ss.add(order)
            ss.add_all(order_items_db)
            ss.add(payment)
            pbar_sub.update()
        pbar.update()

    await ss.flush()


Generate per user:   0%|          | 0/99 [00:00<?, ?it/s]

Order:: 0it [00:00, ?it/s]

In [12]:
import uuid
from db.postgresql.models.order_history import (
    OrderHistory,
    OrderHistoryItems,
    OrderStatus,
    PaymentStatus,
)
from db.postgresql.models.transaction import PaymentTransaction

pbar = tqdm(total=account_amount - 1, desc="Generate per user")
pbar_sub = tqdm(desc="Order:")
async with ss.begin():
    orders = await ss.scalars(sqla.select(OrderHistory))

    for order in orders:


    await ss.flush()

Generate per user:   0%|          | 0/99 [00:00<?, ?it/s]

Order:: 0it [00:00, ?it/s]